# Dashboard SECOP Territoriales

* Leidy Milena Alarcón Castelblanco - lalarconc@ucentral.edu.co
* Otoniel Humberto Castañeda Rodriguez - ocastanedar1@ucentral.edu.co

Instalamos las librerías necesarias 

In [1]:
!pip install dash
!pip install jupyter-dash
!pip install plotly_express
!pip install dash-bootstrap-components

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 1.0MB 9.2MB/s 
     |████████████████████████████████| 3.5MB 18.5MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 37.3MB/s 
     |████████████████████████████████| 358kB 34.7MB/s 
  Created wheel for dash: filename=dash-1.20.0-cp37-none-any.whl size=85834 sha256=0b549ce304bac048ba36c29d8e4de411d756b0c6ab53a0085317eb54f5f71b20
  Stored in directory: /root/.cache/pip/wheels/e6/33/41/ce361d0d1da277d2350f815b10f2ab1e331592d93ef2b07e29
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-cp37-none-any.whl size=1014875 sha256=d4db5520034c8fb36c0472f801fd11f48307fd2044d957f4c4cfe9865c4ef594
  Stored in directory: /root/.cache/pip/wheels/03/a9/c5/dd5815c601b0ede164c223ffd7bafebde716ca57de06ef8aec
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-cp37-none-any.whl size=3540993 sha256=4e74f12ca95fd12acd87b03

Importamos los dataframes procesados

In [2]:
import pandas as pd
import numpy as np
import plotly_express as px
from sklearn.metrics import pairwise_distances_argmin_min
from urllib.request import urlopen

secop_trr_df = pd.read_csv('https://github.com/ocastanedar1/VisualizacionDatos/blob/main/SECOP/secop_territoriales.zip?raw=true', compression='zip')
secop_kpi_df = pd.read_csv('https://raw.githubusercontent.com/ocastanedar1/VisualizacionDatos/main/SECOP/secop_territoriales_kpi.csv', dtype={"Codigo": str})



Ajustamos los tipos de datos

In [3]:

secop_trr_df['Fecha de Inicio del Contrato'] = pd.to_datetime(secop_trr_df['Fecha de Inicio del Contrato'], errors='coerce')
secop_trr_df['Fecha de Fin del Contrato'] = pd.to_datetime(secop_trr_df['Fecha de Fin del Contrato'], errors='coerce')
secop_trr_df['Fecha de Inicio de Ejecucion'] = pd.to_datetime(secop_trr_df['Fecha de Inicio de Ejecucion'], errors='coerce')
secop_trr_df['Fecha de Fin de Ejecucion'] = pd.to_datetime(secop_trr_df['Fecha de Fin de Ejecucion'], errors='coerce')


Creamos un dataframe a partir de las fechas de los contratos (contrato terminados extemporaneso, es decir, que superan 10% de tiempo en días adicionados) 

In [4]:
df_secop_dias = secop_trr_df[~secop_trr_df['Fecha de Fin de Ejecucion'].isna()].copy()
df_secop_dias['Dias dif fecha fin contrato'] =  df_secop_dias['Fecha de Fin de Ejecucion'] - df_secop_dias['Fecha de Fin del Contrato']
df_secop_dias = df_secop_dias[df_secop_dias['Dias dif fecha fin contrato']>pd.Timedelta(days=0)].copy()

df_secop_dias['Dias contrato'] =  df_secop_dias['Fecha de Fin del Contrato'] - df_secop_dias['Fecha de Inicio del Contrato']
df_secop_dias = df_secop_dias[df_secop_dias['Dias contrato']>pd.Timedelta(days=0)].copy()

df_secop_dias['10p del total del contrato'] = df_secop_dias['Dias contrato'] * 0.1

df_secop_dias['Contrato extemporaneo'] = 'N'

df_secop_dias['Contrato extemporaneo'] = np.where(df_secop_dias['Dias dif fecha fin contrato'] >= df_secop_dias['10p del total del contrato'], 'S', 'N')

df_secop_dias = df_secop_dias[df_secop_dias['Contrato extemporaneo'] == 'S'].copy()

df_secop_dias


,Unnamed: 0,Departamento,Localización,Sector,ID Contrato,Estado Contrato,Tipo de Contrato,Modalidad de Contratacion,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,Fecha de Fin de Ejecucion,TipoDocProveedor,Es Pyme,Habilita Pago Adelantado,Liquidación,Obligación Ambiental,Obligaciones Postconsumo,Reversion,Valor del Contrato,Valor de pago adelantado,Valor Facturado,Valor Pendiente de Pago,Valor Pagado,Valor Amortizado,Valor Pendiente de Amortizacion,Valor Pendiente de Ejecucion,EsPostConflicto,Destino Gasto,Origen de los Recursos,Dias Adicionados,Año Fecha de Inicio del Contrato,Mes Fecha de Inicio del Contrato,Dias dif fecha fin contrato,Dias contrato,10p del total del contrato,Contrato extemporaneo
159,586,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",defensa,CO1.PCCNTR.620420,terminado,Prestación de servicios,Selección Abreviada de Menor Cuantía,2018-11-02,2019-04-30,2018-11-02,2019-06-30,NIT,No,No,Si,No,No,No,3.097486e+09,0.0,0.0,3.097486e+09,0.0,0.0,0.0,3.097486e+09,No,Funcionamiento,Distribuido,0.0,2018.0,11.0,61 days,179 days,17 days 21:36:00,S
227,861,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",Salud y Protección Social,CO1.PCCNTR.1421381,terminado,Prestación de servicios,Contratación directa,2020-03-05,2020-08-14,2020-03-05,2020-09-04,Cédula de Ciudadanía,No,No,Si,No,No,No,3.355646e+07,0.0,0.0,3.355646e+07,0.0,0.0,0.0,3.355646e+07,No,Inversión,Distribuido,0.0,2020.0,3.0,21 days,162 days,16 days 04:48:00,S
304,1162,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",Salud y Protección Social,CO1.PCCNTR.726226,Activo,Prestación de servicios,Contratación régimen especial,2019-01-16,2019-04-15,2019-01-16,2019-07-15,Cédula de Ciudadanía,No,No,No,No,No,No,4.230000e+06,0.0,0.0,4.230000e+06,0.0,0.0,0.0,4.230000e+06,No,Funcionamiento,Distribuido,0.0,2019.0,1.0,91 days,89 days,8 days 21:36:00,S
704,2688,Huila,"Colombia, Huila , Neiva",Servicio Público,CO1.PCCNTR.1053702,terminado,Obra,Selección Abreviada de Menor Cuantía,2019-08-15,2019-11-14,2019-08-15,2019-12-11,Cédula de Ciudadanía,Si,No,No,No,No,No,9.752468e+07,0.0,79105799.0,1.841888e+07,79105799.0,0.0,0.0,9.752468e+07,No,Inversión,Distribuido,0.0,2019.0,8.0,27 days,91 days,9 days 02:24:00,S
752,2844,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",Salud y Protección Social,CO1.PCCNTR.813908,terminado,Prestación de servicios,Contratación directa,2019-02-07,2020-01-06,2019-02-07,2020-06-01,Cédula de Ciudadanía,No,No,No,No,No,No,3.390750e+07,0.0,0.0,3.390750e+07,0.0,0.0,0.0,3.390750e+07,No,Inversión,Distribuido,0.0,2019.0,2.0,147 days,333 days,33 days 07:12:00,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263277,1001633,Tolima,"Colombia, Tolima , Ibagué",Servicio Público,CO1.PCCNTR.640043,Activo,Prestación de servicios,Contratación directa,2018-11-09,2018-12-09,2018-11-09,2018-12-31,Cédula de Ciudadanía,Si,No,No,No,No,No,6.666666e+06,0.0,0.0,6.666666e+06,0.0,0.0,0.0,6.666666e+06,No,Inversión,Distribuido,0.0,2018.0,11.0,22 days,30 days,3 days 00:00:00,S
263280,1001647,Valle del Cauca,"Colombia, Valle del Cauca",Cultura,CO1.PCCNTR.809754,terminado,Prestación de servicios,Contratación directa,2019-02-06,2019-05-05,2019-05-02,2019-05-31,Cédula de Ciudadanía,No,No,No,No,No,No,7.807672e+06,0.0,7807672.0,7.807672e+06,0.0,0.0,0.0,7.807672e+06,No,Inversión,Distribuido,0.0,2019.0,2.0,26 days,88 days,8 days 19:12:00,S
263582,1002796,Valle del Cauca,"Colombia, Valle del Cauca",Cultura,CO1.PCCNTR.841439,terminado,Prestación de servicios,Contratación directa,2019-02-18,2019-05-18,2019-05-15,2019-06-14,Cédula de Ciudadanía,No,No,No,No,No,No,5.970572e+06,0.0,5970572.0,5.970572e+06,0.0,0.0,0.0,5.970572e+06,No,Inversión,Distribuido,0.0,2019.0,2.0,27 days,89 days,8 days 21:36:00,S
263645,1003002,Santander,"Colombia, Santander , Bucaramanga",Ambiente y Desarrollo Sostenible,CO1.PCCNTR.325260,Cerrado,Prestación de servicios,Contr

Se agrupan la cantidad de contratos por modalidad

In [5]:
secop_modalidad = secop_trr_df.groupby(by=['Modalidad de Contratacion', 'Año Fecha de Inicio del Contrato', 'Departamento']).count()[['ID Contrato']]
secop_modalidad.reset_index(inplace=True)
secop_modalidad

,Modalidad de Contratacion,Año Fecha de Inicio del Contrato,Departamento,ID Contrato
0,Concurso de méritos abierto,2018.0,Antioquia,8
1,Concurso de méritos abierto,2018.0,Caldas,2
2,Concurso de méritos abierto,2018.0,Cundinamarca,2
3,Concurso de méritos abierto,2018.0,Distrito Capital de Bogotá,146
4,Concurso de méritos abierto,2018.0,Huila,5
...,...,...,...,...
848,Selección abreviada subasta inversa,2021.0,Risaralda,6
849,Selección abreviada subasta inversa,2021.0,Santander,8
850,Selección abreviada subasta inversa,2021.0,Sucre,2
851,Selección abreviada subasta inversa,2021.0,Tolima,5


Se carga el dataframe resultado de cluster realizado con kmeans

In [6]:
secop_clusters_df = pd.read_csv('https://github.com/ocastanedar1/VisualizacionDatos/blob/main/SECOP/secop_clusters.zip?raw=true', compression='zip')
secop_clusters_df

,Unnamed: 0,c0,c1,c2,c3,Valor del Contrato,Dias Adicionados,id_contrato,categoria,label_kameans,clusters_kameans
0,0,-0.751024,0.422201,0.001039,-0.206211,-0.039720,-0.064139,CO1.PCCNTR.2389371,Distrito Capital de Bogotá,0,Cluster 1
1,1,-0.751024,0.422201,0.001039,-0.206211,-0.047511,-0.064139,CO1.PCCNTR.563024,Distrito Capital de Bogotá,0,Cluster 1
2,2,0.365449,-2.257896,0.009412,1.267922,-0.053167,-0.064139,CO1.PCCNTR.648250,Distrito Capital de Bogotá,4,Cluster 5
3,3,4.542877,0.832827,-0.081866,-2.498026,-0.033114,-0.064139,CO1.PCCNTR.495315,Boyacá,3,Cluster 4
4,4,-0.751024,0.422201,0.001039,-0.206211,-0.012831,-0.064139,CO1.PCCNTR.2353332,Distrito Capital de Bogotá,0,Cluster 1
...,...,...,...,...,...,...,...,...,...,...,...
263752,263752,-0.301622,-0.120532,0.002370,-0.015418,-0.049857,-0.064139,CO1.PCCNTR.1655228,Arauca,0,Cluster 1
263753,263753,-0.476197,0.221077,0.022333,-0.047426,-0.039268,-0.064139,CO1.PCCNTR.2178613,Valle del Cauca,0,Cluster 1
263754,263754,-0.562573,0.328447,0.003318,-0.148096,-0.013662,-0.064139,CO1.PCCNTR.2142423,Atlántico,0,Cluster 1
263755,263755,-0.017115,0.076939,-0.000469,0.029460,-0.036365,-0.064139,CO1.PCCNTR.1628557,Atlántico,0,Cluster 1


Se carga el modelo desarrollado con pickle

In [7]:
import pickle

with urlopen('https://github.com/ocastanedar1/VisualizacionDatos/blob/main/SECOP/kmeans_secop.pkl?raw=true') as file:
    pickle_model = pickle.load(file)
    
pickle_model.cluster_centers_[:,1]

array([ 0.15233513,  9.26339659, -2.20949218,  0.62339508, -2.35730469,
        8.86607297,  2.60851651])

Se crea el dashboard con los resultados preprocesados

In [25]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_table

from urllib.request import urlopen
import json

import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


external_stylesheets = [dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

with urlopen('https://raw.githubusercontent.com/ocastanedar1/VisualizacionDatos/main/SECOP/colombia.geo.json') as response:
        counties = json.load(response)

def get_cluster_departamento():
    dep = secop_clusters_df['categoria'].unique()
    grp = secop_clusters_df['clusters_kameans'].unique()

    df_km_gb = secop_clusters_df.groupby(by=['clusters_kameans', 'categoria']).count()[['label_kameans']]
    df_km_gb.reset_index(inplace=True)
    df_km_gb

    for g in grp:
        for d in dep:
            if df_km_gb[(df_km_gb['clusters_kameans'] == g) & (df_km_gb['categoria'] == d)].empty:
                df_km_gb = df_km_gb.append({'clusters_kameans': g, 'categoria': d, 'label_kameans': 0}, ignore_index=True)


    df_km_gb['label_kameans_avg'] = 0

    for d in dep:
        df_km_gb.loc[df_km_gb['categoria'] == d, 'label_kameans_avg'] = df_km_gb[df_km_gb['categoria'] == d]['label_kameans'] / sum(df_km_gb[df_km_gb['categoria'] == d]['label_kameans'])
    
    fig = px.bar(df_km_gb[df_km_gb['label_kameans_avg'].notnull()], x="categoria", y="label_kameans_avg", color="clusters_kameans", title="Proporción de departamentos por cluster")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    return fig

def get_caracteristicas_cont():
    closest, _ = pairwise_distances_argmin_min(pickle_model.cluster_centers_, secop_clusters_df[['c0', 'c1', 'c2', 'c3', 'Valor del Contrato', 'Dias Adicionados']])
    contratos = []
    users=secop_clusters_df['id_contrato'].values
    for row in closest:
        contratos.append(users[row])
    data = {'ID Contrato':  contratos}
    df = pd.DataFrame (data)
    return df.to_dict('records')



app.layout = dbc.Container(
    [
        html.H1("¿Cómo se invierte en contratación en los entes territoriales de Colombia?", style={'color': '#161D26', 'font-size':'2rem'}),
        html.P('El Sistema Electrónico de Contratación Pública (SECOP) es la herramienta que permite gestionar los procesos de contratación que demandan las instituciones territoriales, nacionales y autónomas oficiales, asegurando la trazabilidad y transparencia del uso de los recursos públicos.', style={'color': '#2C3540', 'font-size':'1.2rem'}),
        html.P('Con el fin de analizar el comportamiento de los contratos de las entidades de orden territorial por Departamento durante los años 2018 al 2021, en las siguientes dos secciones se pueden navegar la información desde diferentes perspectivas.', style={'color': '#2C3540', 'font-size':'1.2rem'}),
        html.Br(),
        dbc.Row([
            dbc.Col([
                     dbc.Tabs([
                            dbc.Tab([
                                html.Br(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H2("Indicadores de Desempeño", style={'color': '#161D26', 'font-size':'1.5rem'}),
                                        html.P('Teniendo en cuenta la importancia del uso eficiente de los recursos públicos, se hace necesario medir el rendimiento de los procesos de contratación de los entes territoriales, razón por la cual se proponen los siguientes indicadores:', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                    ]),    
                                ]),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Inversión de recursos por sector y departamento", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Sector: "),
                                            dcc.Dropdown(
                                                id="sector",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_trr_df['Sector'].unique().tolist())
                                                ],
                                                value="Ambiente y Desarrollo Sostenible",
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Año: "),
                                            dcc.Dropdown(
                                                id="ano",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_kpi_df['Año Fecha de Inicio del Contrato'].unique().tolist())
                                                ],
                                                value=2021,
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),     
                                ]),
                                dbc.Row([
                                    dbc.Col(
                                        dcc.Graph(id="secop_dep_map"),
                                    md=9),
                                    dbc.Col([
                                            html.Br(),
                                            html.P('En este indicador permite identificar el volumen de recursos invertidos en entidades públicas territoriales para cada uno de los departamentos, permitiendo visualizar por colores el monto invertido de acuerdo con el sector y año (entre más oscuro menor inversión y entre más claro mayor inversión).', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            html.P('Se puede evidenciar que algunos sectores tienen mayor concentración de recursos, como el servicio público, otros, planeación, salud y protección social, educación nacional y transporte; así como Bogotá, Antioquia y Valle del Cauca, presentan mayor participación en la mayoría de los sectores.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                        ], md=3),
                                ]),
                                html.Br(),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Porcentaje de contratos ejecutados por PYMES", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Año: "),
                                            dcc.Dropdown(
                                                id="ano_pyme",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_kpi_df['Año Fecha de Inicio del Contrato'].unique().tolist())
                                                ],
                                                value=2021,
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),     
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                            html.Br(),
                                            html.P('En relación con los contratistas y dada la importancia que las PYMES tienen en el desarrollo y crecimiento del país, este indicador busca visibilizar la presencia de las PYMES en los procesos de contratación pública territorial.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            html.P('Como se puede observar el indicador se puede consultar por año, no obstante, se evidencia que el porcentaje de participación de las pequeñas y medianas empresas es apenas la cuarta parte del total de inversión.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            
                                        ], md=3),
                                    dbc.Col(
                                        dcc.Graph(id="secop_pyme_pie"),
                                    md=9),
                                ]),
                                html.Br(),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Porcentaje de contratos ejecutados extemporáneamente por departamento (mayor al 10% del tiempo inicialmente contratado)", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Departamento: "),
                                            dcc.Dropdown(
                                                id="departamento_ext",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(df_secop_dias['Departamento'].unique().tolist())
                                                ],
                                                value="Antioquia",
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Año: "),
                                            dcc.Dropdown(
                                                id="ano_ext",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(df_secop_dias['Año Fecha de Inicio del Contrato'].unique().tolist())
                                                ],
                                                value=2020,
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),     
                                ]),
                                dbc.Row([
                                    dbc.Col(
                                        dcc.Graph(id="contratos-extemporaneos-pie"),
                                    md=9),
                                    dbc.Col([
                                            html.Br(),
                                            html.P('Este indicador pretende evidenciar los contratos cuyo tiempo de ejecución fue mayor al tiempo estipulado en el contrato, y que supera el 10% del total del tiempo contratado.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            html.P('Dado que este indicador se mide en términos de las cantidades de contratos, da cuenta sobre que tan cumplidos son los departamentos en el seguimiento y por ende en la ejecución del tiempo del contrato, evidenciando que a nivel general este porcentaje es muy bajo.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                        ], md=3),
                                ]),
                                html.Br(),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Porcentaje de contratos por modalidad de contratación por departamento", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Departamento: "),
                                            dcc.Dropdown(
                                                id="departamento_modalidad",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_modalidad['Departamento'].unique().tolist())
                                                ],
                                                value="Antioquia",
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Año: "),
                                            dcc.Dropdown(
                                                id="ano_modalidad",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_modalidad['Año Fecha de Inicio del Contrato'].unique().tolist())
                                                ],
                                                value=2021,
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),     
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                            html.Br(),
                                            html.P('La contratación pública establece diferentes modalidades de contratación dependiendo del objeto, duración, importancia, impacto y riesgo  del contrato, En este indicador se puede conocer la distribución de las diferentes modalidades de contratación por departamento y año.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            html.P('A nivel general, se evidencia una mayor participación de la modalidad contratación directa para los años 2020 y 2021, dada la declaración de urgencia manifiesta causada por la pandemia Covid-19, que son una de las causas del uso de esta modalidad y que es independiente de la cuantía del contrato suscrito.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            
                                        ], md=3),
                                    dbc.Col(
                                        dcc.Graph(id="secop_modalidad_bar"),
                                    md=9),
                                ]),
                                html.Br(),
                            ], label="Indicadores"),
                            dbc.Tab([
                                html.Br(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H2("Identificación de las Características de Contratación Territorial en Colombia", style={'color': '#161D26', 'font-size':'1.5rem'}),
                                        html.P('Con el fin de clasificar los Departamentos de acuerdo con las características de contratación se aplicó un modelo de agrupamiento (Cluster Kmeans), para determinar las semejanzas entre las diferentes condiciones de contratación en los entes territoriales.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                        html.P('Dado que se contaban con varias variables cualitativas se aplicó un método de reducción de dimensionalidad (Análisis de Correspondencias Múltiples), que junto con las variables monto de  contrato y días adicionados, se incorporaron a la técnica multivariante cluster, con los siguientes resultados:', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                    ]),    
                                ]),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Clusters por variables", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Variable A: "),
                                            dcc.Dropdown(
                                                id="var_a",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_clusters_df[['c0', 'c1', 'c2', 'c3', 'Valor del Contrato', 'Dias Adicionados']].columns.tolist())
                                                ],
                                                value="c0",
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),
                                    dbc.Col([
                                        dbc.FormGroup([
                                            dbc.Label("Variable B: "),
                                            dcc.Dropdown(
                                                id="var_b",
                                                options=[
                                                    {"label": col, "value": col} for col in sorted(secop_clusters_df[['c0', 'c1', 'c2', 'c3', 'Valor del Contrato', 'Dias Adicionados']].columns.tolist())
                                                ],
                                                value="c1",
                                                searchable=False,
                                                clearable=False,
                                            ),
                                        ]),
                                    ]),     
                                ]),
                                dbc.Row([
                                    dbc.Col(
                                        dcc.Graph(id="clusters_fig"),
                                    md=9),
                                    dbc.Col([
                                            html.Br(),
                                            html.P('En esta gráfica se puede visualizar la distribución de los cluster (colores) en pares de variables, agrupando de acuerdo a características de los contratos de entidades territoriales a nivel nacional en 7 grupos.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            
                                        ], md=3),
                                ]),
                                html.Br(),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Clusters por departamentos", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                            html.Br(),
                                            html.P('En esta gráfica se puede observar la participación de cada uno de los cluster por departamento, observando que el cluster 1 presencia en la mayoría de los departamentos del país, lo que nos permite concluir que existen similitudes en las características de contratación en Colombia.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            
                                        ], md=3),
                                    dbc.Col(
                                        dcc.Graph(id="secop_cluster_departamento", figure=get_cluster_departamento()),
                                    md=9),
                                ]),
                                html.Br(),
                                html.Hr(),
                                dbc.Row([
                                    dbc.Col([
                                        html.H3("Contratos más cercanos a cada uno de los clusters", style={'color': '#161D26', 'font-size':'1.2rem'}),
                                    ]),    
                                ]),
                                dbc.Row([
                                    dbc.Col(
                                        dash_table.DataTable(
                                            id='table-contratos',
                                            columns=[{'id': c, 'name': c} for c in ['ID Contrato']],
                                            data=get_caracteristicas_cont(),
                                        ), md=9),
                                    dbc.Col([
                                            html.Br(),
                                            html.P('Con el fin de establecer las características de los agrupamientos, en la tabla se relacionan los contratos (ID) que describen mejor cada uno de los cluster, información que puede ser validada en los registros del SECOP.', style={'color': '#2C3540', 'font-size':'1.1rem'}),
                                            
                                        ], md=3),
                                ]),
                                html.Br(),
                            ], label="Modelo"),
                        ])
                ]),
        ]),
        html.Br(),
        html.Hr(),
        html.P("* Fuente: El Sistema Electrónico para la Contratación Pública – SECOP", style={'color': '#2C3540', 'font-size':'0.7rem'}),
        html.P("Visualización de Datos – Maestría en Analítica de Datos – Universidad Central", style={'color': '#2C3540', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        html.P("Leidy Milena Alarcón - lalarconc@ucentral.edu.co", style={'color': '#2C3540', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        html.P("Otoniel Humberto Castañeda - ocastanedar1@ucentral.edu.co", style={'color': '#2C3540', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        html.P("2021", style={'color': '#2C3540', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        
    ],
    fluid=True,
    style={'background':'#D4DADC', 'padding':'4rem'},
)

#########################################################TAB 1###########################################################

@app.callback(
    Output('secop_pyme_pie', 'figure'),
    [Input('ano_pyme', 'value')])
def secop_kpi_pyme(ano_pyme):
    df = secop_trr_df[secop_trr_df['Año Fecha de Inicio del Contrato'] == ano_pyme][['Valor del Contrato', 'Es Pyme']]
    fig = px.pie(df, values='Valor del Contrato', names='Es Pyme')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    return fig

@app.callback(
    Output('secop_modalidad_bar', 'figure'),
    [Input('departamento_modalidad', 'value'),
     Input('ano_modalidad', 'value')])
def secop_modalidad_f(departamento_modalidad, ano_modalidad):
    df = secop_modalidad[(secop_modalidad['Año Fecha de Inicio del Contrato'] == ano_modalidad) & (secop_modalidad['Departamento'] == departamento_modalidad)]
    fig = px.bar(df, x='Modalidad de Contratacion', y='ID Contrato')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    return fig

@app.callback(
    Output('secop_dep_map', 'figure'),
    [Input('sector', 'value'),
     Input('ano', 'value')])
def secop_dep_map(sector='Ambiente y Desarrollo Sostenible', ano='2021'):

    sector = 'Sector de ' + sector
    df = secop_kpi_df[(secop_kpi_df['Año Fecha de Inicio del Contrato'] == ano) & (secop_kpi_df[sector] != 0)][['Año Fecha de Inicio del Contrato', 'Departamento', sector, 'Codigo']].copy()

    fig = px.choropleth_mapbox(df, geojson=counties, locations='Codigo', color=sector,
                            mapbox_style="carto-positron",
                            range_color=(0, max(df[(df['Año Fecha de Inicio del Contrato'] == ano) & (df[sector] != 0)][sector])),
                            zoom=4, center = {"lat": 3.7853839512796505, "lon": -73.25720435955333},
                            opacity=0.5,
                            labels={sector:'Millones de pesos'}
                            )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    
    return fig

@app.callback(
    Output('contratos-extemporaneos-pie', 'figure'),
    [Input('departamento_ext', 'value'),
     Input('ano_ext', 'value')])
def secop_ext_pie(departamento_ext, ano_ext):    

    labels = ['Extemporaneos','A tiempo']
    values = [len(df_secop_dias[(df_secop_dias['Departamento'] == departamento_ext) & (df_secop_dias['Año Fecha de Inicio del Contrato'] == ano_ext)]), len(secop_trr_df[(secop_trr_df['Departamento'] == departamento_ext) & (secop_trr_df['Año Fecha de Inicio del Contrato'] == ano_ext)])]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    return fig

###########################################################TAB 2###################################################################

@app.callback(
    Output('clusters_fig', 'figure'),
    [Input('var_a', 'value'),
     Input('var_b', 'value')])
def get_clusters_fig(var_a, var_b):
    df = secop_clusters_df[['c0', 'c1', 'c2', 'c3', 'Valor del Contrato', 'Dias Adicionados']]

    fig = px.scatter(secop_clusters_df, x=var_a, y=var_b, color="clusters_kameans")
    fig.add_scatter(x=pickle_model.cluster_centers_[:,df.columns.tolist().index(var_a)], y=pickle_model.cluster_centers_[:,df.columns.tolist().index(var_b)], name = "Cluster", mode='markers', marker={'symbol': 'diamond-wide', 'size': 20, 'color': ['blue', 'red', 'green', 'purple', 'yellow', 'orange',  'pink'], 'line':None})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    return fig


app.run_server(mode='inline', debug=True, port=8051)



/usr/local/lib/python3.7/dist-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



<IPython.core.display.Javascript object>

In [26]:
app.run_server(debug=True)

Dash app running on:


/usr/local/lib/python3.7/dist-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



<IPython.core.display.Javascript object>